In [1]:
import torch
import torch.nn as nn 
from torchvision import models 
from torchsummary import summary
from torchvision import transforms, datasets
import numpy as np
from tqdm import tqdm
from functools import partial

tqdm = partial(tqdm, position=0, leave=True)

In [2]:
train_transform = transforms.Compose(transforms= [
    transforms.RandomSizedCrop(32, padding=4),
    transforms.Resize((32,32)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.48216, 0.44653], std=[0.2023, 0.1994, 0.2010])
    
])
valid_transforms = transforms.Compose(transforms=[
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.48216, 0.44653], std=[0.2023, 0.1994, 0.2010])
   
])

In [3]:
batch_size=128

trainset = datasets.CIFAR10(root='C:/CIFAR10', train=True, download=True, transform=train_transform)
validset = datasets.CIFAR10(root='C:/CIFAR10', train=False, download=True, transform=valid_transforms)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = models.resnet18(pretrained=True)
        for param in self.model.parameters():
            param.requires_grad = False
        self.model.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1, bias=False)
        self.model.maxpool = nn.Identity()
        
        self.model.fc = nn.Linear(self.model.fc.in_features, 10)

    
    def forward(self, x):
        return self.model(x)

In [5]:
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
valid_loader = torch.utils.data.DataLoader(validset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

pred_to_class = {i:cl for i,cl in enumerate(classes)}

In [6]:
next(iter(valid_loader))

In [6]:
model = Model()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.9)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[25,40])

criterion = nn.CrossEntropyLoss()


In [7]:
x,y = next(iter(train_loader))
x.shape, y.shape

In [9]:
DEVICE='cuda'
model.to(DEVICE)
def train(model, train_loader, optimizer, loss_fn, device=DEVICE):
    model.train()
    correct = 0
    train_loss = 0

    for _, data in tqdm(enumerate(train_loader), total=len(train_loader)):
        
        data, target = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()

        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        train_loss += loss.detach().item()
        loss.backward()
        optimizer.step()

        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()


    # train_loss.append(train_loss * 1.0 / len(train_loader.dataset))
    # train_acc.append(correct * 100.0 / len(train_loader.dataset))

    print(
        f"Avg Train loss = {train_loss * 1.0 / len(train_loader.dataset)}, Train Accuracy : {100.0 * correct / len(train_loader.dataset)}"
    )

def evaluate(model, valid_loader, loss_fn, device=DEVICE):
    model.eval()
    correct = 0
    valid_loss = 0
    
    with torch.no_grad():
        for _, data in tqdm(enumerate(valid_loader), total=len(valid_loader)):
            data, target = data[0].to(device), data[1].to(device)
            output = model(data)
            valid_loss += loss_fn(output, target, reduction="sum").item()
            pred = output.argmax(dim=1, keepdims=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    
    valid_loss /= len(valid_loader.dataset) * 1.0
    valid_loss.append(valid_loss)
    # valid_acc.append(100.0 * correct / len(valid_loader.dataset))
    
    print(
        f"Avg Valid loss = {valid_loss}, Valid Accuracy : {100.0 * correct / len(valid_loader.dataset)}"
    )

In [ ]:
for epoch in range(20):
    print(f'Epoch {epoch + 1}')
    train(model, train_loader, optimizer, criterion)
    evaluate(model, valid_loader, criterion)
    scheduler.step()
    

Epoch 1


In [10]:
# model.load_state_dict(torch.load('E:\TSAI\EMLO\EMLO-session2\models\emlo_session2_model.pt'))
# model.eval()

In [2]:
from PIL import Image

img = Image.open("E:\\TSAI\\EMLO\\EMLO-session2\\test_imgs\\car_img.jpg")
img.load()
# data = np.asarray(img, dtype="int32')
# data.shape

In [3]:
type(img)

PIL.JpegImagePlugin.JpegImageFile

In [40]:
data = valid_transforms(img)

In [41]:
pred_to_class

{0: 'plane',
 1: 'car',
 2: 'bird',
 3: 'cat',
 4: 'deer',
 5: 'dog',
 6: 'frog',
 7: 'horse',
 8: 'ship',
 9: 'truck'}

In [42]:
data.shape, data.unsqueeze(0).shape

(torch.Size([3, 32, 32]), torch.Size([1, 3, 32, 32]))

In [43]:
output = model(data.unsqueeze(0)).detach()
pred_idx = output.numpy().argmax()

In [44]:
pred_proba = torch.exp(torch.nn.functional.log_softmax(output, dim=1))
pred_proba

tensor([[0.0719, 0.0478, 0.0772, 0.0816, 0.1175, 0.0697, 0.1070, 0.1443, 0.1978,
         0.0853]])

In [46]:
pred_to_class[pred_idx]

'ship'